In [3]:
import pandas as pd
import numpy as np
import torch

In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [9]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [69]:
X_train,X_test,Y_train,Y_test=train_test_split(df.iloc[:,1:],df.iloc[:,:1],test_size=0.15)

In [70]:
label_encoder=LabelEncoder()
scaler=StandardScaler()

In [71]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
Y_train=label_encoder.fit_transform(Y_train.to_numpy().flatten())
Y_test=label_encoder.transform(Y_test.to_numpy().flatten())

In [72]:
X_train=torch.from_numpy(X_train)
Y_train=torch.from_numpy(Y_train)
X_test=torch.from_numpy(X_test)
Y_test=torch.from_numpy(Y_test)

In [99]:
def binary_cross_entrophy_loss(y_pred,y_true):
    epsilon=1e-9
    y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
    loss=-torch.mean(y_true*torch.log(y_pred)+(1-y_true)*torch.log(1-y_pred))
    return loss

In [100]:
class Model:
    def __init__(self,input_size:int):
        self.weights=torch.rand(input_size,1,requires_grad=True,dtype=torch.double)
        self.bias=torch.rand(1,requires_grad=True)
    def forward(self,input_):
        z=torch.matmul(input_,self.weights)+self.bias
        y_pred=torch.sigmoid(z)
        return y_pred

In [101]:
model=Model(X_train.shape[1])

In [98]:
Y_pred=model.forward(X_train)

In [ ]:
epochs=10000
learning_rate=0.001
for i in range(epochs):
    y_pred=model.forward(X_train)
    loss=binary_cross_entrophy_loss(y_pred,Y_train.reshape(-1,1))
    loss.backward()
    with torch.no_grad():
        model.weights-=learning_rate*model.weights.grad
        model.bias-=learning_rate*model.bias.grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    if i%100==0:
        print(f'Loss: {loss}')

In [121]:
y_pred=model.forward(X_test)

In [128]:
y_pred=y_pred.flatten()>0.5

In [133]:
torch.mean((y_pred.float()==Y_test).float())

tensor(0.9884)